**Exploratory Data Analysis**

In [1]:
# Loading necesarry libaries
import pandas as pd

In [2]:
# Loading the dataset
df = pd.read_csv("flight_data.csv")

First we'll need to transform the variables into the correct datatype and clean the data

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Search Date         900 non-null    object
 1   Airline             900 non-null    object
 2   Flight Number       900 non-null    int64 
 3   Source City         900 non-null    object
 4   Departure Time      900 non-null    object
 5   Number of Stops     900 non-null    int64 
 6   Arrival Time        900 non-null    object
 7   Destination City    900 non-null    object
 8   Duration (Minutes)  900 non-null    int64 
 9   Price               900 non-null    object
dtypes: int64(3), object(7)
memory usage: 70.4+ KB


In [24]:
# Convert columns to datetime
df['Search Date'] = pd.to_datetime(df['Search Date']).dt.date
df['Departure Time'] = pd.to_datetime(df['Departure Time']).dt.strftime('%Y-%m-%d %H:%M:%S')
df['Arrival Time'] = pd.to_datetime(df['Arrival Time']).dt.strftime('%Y-%m-%d %H:%M:%S')
df['Departure Time'] = pd.to_datetime(df['Departure Time']).dt.floor('S')
df['Arrival Time'] = pd.to_datetime(df['Arrival Time']).dt.floor('S')


# Convert Number of Stops to categorical
df['Source City'] = df['Source City'].astype('category')
df['Destination City'] = df['Source City'].astype('category')
df['Airline'] = df['Airline'].astype('category')
df['Flight Number'] = df['Flight Number'].astype('category')

df['Number of Stops'] = df['Number of Stops'].astype('category')

# Price conversion: remove the dollar sign and convert to float
df['Price'] = df['Price'].str.replace('$', '').astype(float)

# Check the new dtypes
print(df.dtypes)


Search Date                   object
Airline                     category
Flight Number               category
Source City                 category
Departure Time        datetime64[ns]
Number of Stops             category
Arrival Time          datetime64[ns]
Destination City            category
Duration (Minutes)             int64
Price                        float64
dtype: object


/var/folders/yk/c34vj_zs38gcc2_lmpzdbdn40000gn/T/ipykernel_3114/4029201694.py:5: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['Departure Time'] = pd.to_datetime(df['Departure Time']).dt.floor('S')
/var/folders/yk/c34vj_zs38gcc2_lmpzdbdn40000gn/T/ipykernel_3114/4029201694.py:6: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['Arrival Time'] = pd.to_datetime(df['Arrival Time']).dt.floor('S')
